# Medical Cost Personal Datasets

# Data Information


About Dataset
Context
Machine Learning with R by Brett Lantz is a book that provides an introduction to machine learning using R. As far as I can tell, Packt Publishing does not make its datasets available online unless you buy the book and create a user account which can be a problem if you are checking the book out from the library or borrowing the book from a friend. All of these datasets are in the public domain but simply needed some cleaning up and recoding to match the format in the book.

Content
Columns

age: age of primary beneficiary

sex: insurance contractor gender, female, male

bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

children: Number of children covered by health insurance / Number of dependents

smoker: Smoking

region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

charges: Individual medical costs billed by health insurance


In [ ]:
%time
import scipy.stats as st
import pylab
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
#loading the dataset

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
spark = SparkSession.builder.appName("example").config("spark.log.level", "DEBUG").getOrCreate()
import warnings
warnings.filterwarnings("ignore")

from pyspark.ml.feature import QuantileDiscretizer
from pyspark.sql.functions import col, lit, concat_ws
from pyspark.sql.functions import when ,concat,lit , round, mean ,median
from pyspark.sql import functions as F

In [ ]:
df=spark.read.csv("/Users/nithinkumar/Downloads/insurance.csv",header=True,inferSchema=True)

# Data Preprocessing

In [ ]:
#checking the shape of the dataset
print("Remaining columns:", len(df.columns))
print("Number of rows:", df.count())

In [ ]:
#checking the columns of the dataset

df.columns

In [ ]:
#null values count
from pyspark.sql.functions import col

# Assuming df is your DataFrame
null_counts = []

# Iterate over all columns in the DataFrame
for col_name in df.columns:
    # Count the number of null values for each column
    null_count = df.where(col(col_name).isNull()).count()
    # Append the result to the list
    null_counts.append((col_name, null_count))

# Display the null counts for each column
for col_name, count in null_counts:
    print(f"Column : '{col_name}' has {count} null values.")


In [ ]:
#column data types
df.printSchema()

In [ ]:
#dulicate values
df=df.dropDuplicates()
#checking the shape of the dataset
print("Remaining columns:", len(df.columns))
print("Number of rows:", df.count())

In [ ]:
df_pd=df.describe().toPandas()
df_pd

In [ ]:
numerical_cols=[]
catorical_cols=[]
for cols in df.columns:
    (catorical_cols.append(cols) if df.select(cols).dtypes[0][1]=='string' else numerical_cols.append(cols))
df_num=df.select(numerical_cols)
df_cat=df.select(catorical_cols)

In [ ]:
df_num.columns

In [ ]:
df_cat.columns

In [ ]:
df = df.select([col(column).alias(column.strip()) for column in df.columns])

In [ ]:
df_pd=df.toPandas()
df_pd.corr()*100

In [ ]:
df_pd.head()

In [ ]:
def PiePlot(Col_name):
    df_modified=df.groupBy(col(Col_name).alias(Col_name)).count()
    df_modified=df_modified.toPandas()
    plt.figure(figsize=(3,3))
    plt.pie(df_modified['count'],labels=df_modified[Col_name],autopct='%.2f%%',radius=1
           ,wedgeprops=dict(width=.5)
            ,pctdistance=.75
           )

In [ ]:
def barh(Col_name,x_Col_name,y_label,x_label,type_bar='H'):
    plt.figure(figsize=(5,5))
    df_modified=df.groupBy(col(Col_name).alias(Col_name)).count()
    df_modified=df_modified.orderBy('count',ascending=False)
    df_modified=df_modified.toPandas()
    df_modified['count']=df_modified['count']/sum(df_modified['count'])*100
    df_modified=df_modified.head(10)
    if type_bar=='H':
        sns.barplot(y=df_modified[Col_name],x=df_modified[x_Col_name]);
        for i, val in enumerate(df_modified.index):
            y = df_modified['count'].loc[val].sum()
            #print(y.dtype)
            plt.text( y+1, i,str(int(y))+'%', ha="center",fontsize = 15,         color ='black')
    else :
        sns.barplot(x=df_modified[Col_name],y=df_modified[x_Col_name]);
        for i, val in enumerate(df_modified.index):
            y = df_modified['count'].loc[val].sum()
            plt.text(i, y ,str(int(y))+'%' , ha="center",fontsize = 15,         color ='black')

    plt.ylabel(y_label,fontsize=10);
    plt.xlabel(x_label,fontsize=10);
    # Add legend outside the plot
    plt.legend(bbox_to_anchor=(1, 1));
    plt.tight_layout();

In [ ]:
def Bins(n,Col_name,Extra_Filter,x_label,y_label,plot):

    quantileDiscretizer=QuantileDiscretizer(numBuckets=n,inputCol=Col_name,outputCol='Col_bin')
    model=quantileDiscretizer.fit(df)
    df_Age_with_bins=model.transform(df)
    bin_edges = model.getSplits()

    bin_edges.pop()
    bin_edges.pop(0)
    bin_edges.insert(0,float(df_Age_with_bins.select(F.min(Col_name)).collect()[0][0]))
    if bin_edges[-1]!=float(df_Age_with_bins.select(F.max(Col_name)).collect()[0][0]):
        bin_edges.append(float(df_Age_with_bins.select(F.max(Col_name)).collect()[0][0]))
    bin_edges=[ int(i) for i in bin_edges]
    #print(len(bin_edges))
    if len(bin_edges)==5:
        return Bins(n+1,Col_name,Extra_Filter,x_label,y_label,plot)
    
    # Create a new column based on bin edges
    # Create a new column based on bin edges
    df_Age_with_bins = model.transform(df).withColumn(
        'Col_bin',
    #    when(df['Age'] <= bin_edges[0], lit(str(bin_edges[0]))).
        when((df[Col_name] >=bin_edges[0]) & (df[Col_name] < bin_edges[1]),\
              concat(lit(str(bin_edges[0])),lit('-'),lit(str(bin_edges[1]))))
        .when((df[Col_name] >= bin_edges[1]) & (df[Col_name] < bin_edges[2]), \
              concat(lit(str(bin_edges[1])),lit('-'),lit(str(bin_edges[2]))))
        .when((df[Col_name] >= bin_edges[2]) & (df[Col_name] < bin_edges[3]),\
              concat(lit(str(bin_edges[2])),lit('-'),lit(str(bin_edges[3]))))
        .when((df[Col_name] >= bin_edges[3]) & (df[Col_name] < bin_edges[4]), \
              concat(lit(str(bin_edges[3])),lit('-'),lit(str(bin_edges[4]))))
        .otherwise(concat(lit(str(bin_edges[4])),lit('-'),lit(str(bin_edges[5]))))  # Handle values greater than the last bin edge
    )
    df_Age_with_bins = df_Age_with_bins.groupBy('Col_bin',Extra_Filter)\
    .agg(round(median("charges")).alias("Loan_Amount"))
    df_Age_with_bins=df_Age_with_bins.orderBy('Loan_Amount')
    df_Age_with_bins=df_Age_with_bins.toPandas()
    plt.figure(figsize=(10,3));

    if plot=='point':
        sns.pointplot(x=df_Age_with_bins.Col_bin,y=df_Age_with_bins.Loan_Amount
                     ,hue=df_Age_with_bins[Extra_Filter]);
    elif plot=='rel':
        sns.relplot(x=df_Age_with_bins.Col_bin,y=df_Age_with_bins.Loan_Amount
                     ,hue=df_Age_with_bins[Extra_Filter]);    
    elif plot=='scatter':
        sns.scatterplot(x=df_Age_with_bins.Col_bin,y=df_Age_with_bins.Loan_Amount
                     ,hue=df_Age_with_bins[Extra_Filter]) ;
    else:
        sns.lineplot(x=df_Age_with_bins.Col_bin,y=df_Age_with_bins.Loan_Amount
                     ,hue=df_Age_with_bins[Extra_Filter]);
    plt.xlabel(x_label,fontsize=10);
    plt.ylabel(y_label,fontsize=10);
    # Add legend outside the plot
    #plt.legend(bbox_to_anchor=(1, 1));
    plt.xticks(fontsize=10);
    plt.yticks(fontsize=10);

In [ ]:
df_pd.head()

# Understanding of Data

In [ ]:
PiePlot('sex')

In [ ]:
PiePlot('smoker')

In [ ]:
barh('children','count','childrens','Number of Customers','K');

In [ ]:
barh('region','count','region','Number of Customers','K');

In [ ]:
df.columns

In [ ]:
Bins(5,'age','sex','Age-Bin','charges','line')

In [ ]:
Bins(5,'age','smoker','Age-Bin','charges','scatter')

In [ ]:
Bins(5,'age','children','Age-Bin','charges','point')

In [ ]:
Bins(5,'age','region','Age-Bin','region','rel');

In [ ]:
Bins(5,'bmi','sex','Bmi-Bin','Charges','line')

In [ ]:
Bins(5,'bmi','smoker','Bmi-Bin','Charges','scatter')

In [ ]:
Bins(5,'bmi','children','Bmi-Bin','Charges','point')

In [ ]:
Bins(5,'bmi','region','Bmi-Bin','region','rel');

# Based on the information provided, here are some key points for Exploratory Data Analysis (EDA):

Demographic Analysis:
Gender Distribution:

Almost equal representation of customers from both genders.
Smoker vs. Non-Smoker:

The majority of customers are non-smokers (80%), while 20% are smokers.
Smokers tend to have higher charges, except in the 24-34 age group where charges are comparable between smokers and non-smokers.
Children Distribution:

42% of customers have no children.
For customers aged 18-24, those with 4 children are more than those with 2 children.
In the 34-44 age group, customers with 4 children outnumber those with 5 children.
Age-Based Analysis:
Age Groups:

Charges are consistent across regions, indicating that pricing is primarily age-dependent rather than region-dependent.
Charges are notably lower for the 24-34 and 44-54 age groups.
Age and Gender:

Females generally incur higher charges than males, except in the 24-34 age group where charges are equal.
Smokers in the 24-34 age group have lower charges compared to other age bins.
BMI Analysis:
BMI and Gender:

Females are charged more for BMI in the range of 15-28, while males are charged more for BMI above 28.
BMI and Region:

Southeast charges more for BMI in the range of 15-28, and the Northeast charges more for BMI above 28.
Children Analysis:
Children and Age:

For customers aged 15-25, those with 4 children are charged more than those with 2 children.
In the 25-28 age group, charges follow the order: 3>5>2>1>4>0.
Children and Region:

For customers with BMI in the range of 15-28, the Southeast charges more, while for BMI above 32, the Southeast charges less.

Overall:

No Children Data for 15-25 Age Group:

No data available for customers with no children in the 15-25 age group.
Consistent Charges in Certain Age Groups:

Charges remain consistent for certain age groups, irrespective of the number of children or region.
Smoker Charges Anomaly in 24-34 Age Group:

An anomaly where smokers in the 24-34 age group have lower charges compared to other age groups.

The customers from both gender is almost equal
The customers having less Non-smoker(80%) compare to smoker(20%)
The customers are more with no childrens.
42 % - No children
24 % - 1 children
17 % - 2 children
11 % - 3 children
1 % - 4 children
1 % - 5 children
Please refer the below regions with there percentage.
27% - SounthEast , 24%- other region 
For Every age Female charges little more then male but at the 24- 34 both changed same.
The charges on the smoker is more then non-smoker but 24-34 the charges are less compared to othr age bins
At the Ages from 18-24:
1.For the children with 4 childrens are more then 2 childrens
2.but on same age for 3 and 5 charges same prices  and childrens betwee 0 and 1 are both are sam
At the Ages from 34-44:
1.For the children with 4 childrens are more then 5 childrens
2.but on same age with 1,2,3,0 are charging same price.
At the 24-34 and 44-54 ages
They are charging same for all.
REgion :
for all the region price depends on Age not region to much

BMI:

For the Female they are charging more for 15-28 BMI
For the Male they are charging more above 28 BMI
Smoker:
from 15-28  they are changing less then BMI which is more 28+
Childrens:
15-25:1.We don;t have any data for customer with no childrens?
		2.For the other customer with who having 5,3,1 charging more.
		3.Customer with 4 childrens charing more in this.
25-28: The follows charges for childrens 3>5>2>1>4>0
28-32: The follows charges for childrens 0,3>4>2>1>5
32-35:The follows charges for childrens 4>3,2,0>1>5
35-53: The follows charges for childrens 3>0,2,4>1,5
Region:
15-28:  Southeast is more charged and above this bmi northeast charges mored
for 32+ the southeast is charging less
25-28 : north east is less


# Model

In [ ]:
#result_df=df
result_df1=result_df

In [ ]:
df=result_df1

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

In [ ]:
def One_Hot_Ender(inputCol_name,OutputCol_name,alias_name,df):
    from pyspark.sql.functions import collect_list
    df_values={}
    indexer=StringIndexer(inputCol=inputCol_name,outputCol=OutputCol_name)
    indexer_model = indexer.fit(df)
    df = indexer_model.transform(df)
    indexed_labels = indexer_model.labels
    for label, index in zip(indexed_labels, range(len(indexed_labels))):
        df_values[label]=index
#    df=df.drop(inputCol_name)
   # df=df.withColumnRenamed(OutputCol_name,inputCol_name)
    return df,df_values

In [ ]:
df,df_values=One_Hot_Ender('sex','sexs','gen',df)
df_values

In [ ]:
df.show(5)

In [ ]:
df=df.drop('sex')

In [ ]:
df=df.withColumnRenamed('sexs','sex')

In [ ]:
df.show(5)

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [ ]:
df,df_values=One_Hot_Ender('region','regions','reg',df)
df_values

In [ ]:
df.show(5)

In [ ]:
df=df.drop('region')
df=df.withColumnRenamed('regions','region')

In [ ]:
df.show(5)

In [ ]:
ordinal_mapping = df.select('smoker').distinct().rdd.flatMap(lambda x: x).collect()

In [ ]:
ordinal_mapping

In [ ]:
ordinal_mapping=dict(zip(ordinal_mapping,[0,1]))
ordinal_encoder=udf(lambda category:ordinal_mapping[category],IntegerType())
df = df.withColumn("smokers", ordinal_encoder(col("smoker")))
#df=df.drop("smoker")

In [ ]:
df.show(5)

In [ ]:
df=df.drop("smoker")

In [ ]:
df.show(5)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
selected_columns=list(df.columns)
vector_Assembler=VectorAssembler(inputCols=df.columns,outputCol='features')
df_assembled=vector_Assembler.transform(df).select('features')
#print(df_assembled.show(5))
correlation_matrix = Correlation.corr(df_assembled, "features").head()
#print(correlation_matrix)
correlation_matrix = correlation_matrix[0].toArray()[selected_columns.index('charges')]
columns = ['Column', 'Correlation with Loan Amount']
data = dict(zip(selected_columns, correlation_matrix))
for column, correlation in data.items():
    print(f"{column}: {correlation*100:.2f}")

In [ ]:
x=df.drop(*['charges']).toPandas()
y=df.select('charges').toPandas()

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=.8,random_state=33)

In [ ]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
x_train.corr()*100

In [ ]:
def correlation(dataset,threhold):
    col_corr=set()
    corr_matrix=dataset.corr()*100
    #print(corr_matrix)
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i,j])>threhold:
                colname=corr_matrix.columns[i]
                #print(colname)
                col_corr.add(colname)
    return col_corr

In [ ]:
correlation(x_train,70)

# Feature Scaling or Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [ ]:
scaler

In [ ]:
x_test.columns

In [ ]:
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

# BOX PLOTS TO UNDERSTAND EFFECT OF STANDARD SCALER

In [ ]:
plt.figure(figsize=(10,3))
sns.boxplot(x_test);

In [ ]:
plt.figure(figsize=(10,3))
sns.boxplot(x_test_scaled);

# Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error, mean_squared_error

In [ ]:
from sklearn.linear_model import LinearRegression
regression=LinearRegression()
regression.fit(x_train_scaled,y_train)
y_pred=regression.predict(x_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
score=r2_score(y_pred,y_test)
print(mae,mape,mse,score)
plt.scatter(y_pred,y_test);

In [ ]:
from sklearn.linear_model import Lasso
regression=Lasso()
regression.fit(x_train_scaled,y_train)
y_pred=regression.predict(x_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
score=r2_score(y_pred,y_test)
print(mae,mape,mse,score)
plt.scatter(y_pred,y_test);

In [ ]:
from sklearn.linear_model import LassoCV
regression=LassoCV(cv=5)
regression.fit(x_train_scaled,y_train)
y_pred=regression.predict(x_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
score=r2_score(y_pred,y_test)
print(mae,mape,mse,score)
plt.scatter(y_pred,y_test);

In [ ]:
from sklearn.linear_model import Ridge
regression=Ridge()
regression.fit(x_train_scaled,y_train)
y_pred=regression.predict(x_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
score=r2_score(y_pred,y_test)
print(mae,mape,mse,score)
plt.scatter(y_pred,y_test);

In [ ]:
from sklearn.linear_model import RidgeCV
regression=RidgeCV(cv=5)
regression.fit(x_train_scaled,y_train)
y_pred=regression.predict(x_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
score=r2_score(y_pred,y_test)
print(mae,mape,mse,score)
plt.scatter(y_pred,y_test);

In [ ]:
regression.get_params()

In [ ]:
from sklearn.linear_model import ElasticNet
regression=ElasticNet()
regression.fit(x_train_scaled,y_train)
y_pred=regression.predict(x_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
score=r2_score(y_pred,y_test)
print(mae,mape,mse,score)
plt.scatter(y_pred,y_test);

In [ ]:
from sklearn.linear_model import ElasticNetCV
regression=ElasticNetCV(cv=5)
regression.fit(x_train_scaled,y_train)
y_pred=regression.predict(x_test_scaled)
mae=mean_absolute_error(y_test,y_pred)
mape=mean_absolute_percentage_error(y_pred,y_test)
mse=mean_squared_error(y_pred,y_test)
score=r2_score(y_pred,y_test)
print(mae,mape,mse,score)
plt.scatter(y_pred,y_test);

# Pickle the Ml models , prepressin model Standscaler

In [ ]:
scaler

In [ ]:
import pickle

In [ ]:
pickle.dump(scaler,open('scaler.pkl','wb'))

In [ ]:
pickle.dump(regression,open('ridge.pkl','wb'))

In [ ]:
import requests

def get_state_from_ip(ip):
    try:
        response = requests.get(f'https://ipinfo.io/{ip}/json', timeout=100)
        data = response.json()
        city = data.get('city', 'N/A')
        region = data.get('region', 'N/A')
        latitude = data.get('latitude', 'N/A')
        longitude = data.get('longitude', 'N/A')
        postal = data.get('postal', 'N/A')
        return state,city,region,latitude,longitude,postal

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

ip_address = '49.37.232.202'
get_state_from_ip(ip_address)
#state
#if state:
   # print(f"The state for IP address {ip_address} is: {state}")
#else:
    #print(f"Unable to determine the state for IP address {ip_address}")


In [ ]:
  "ip": "49.14.101.16",
  "city": "Delhi",
  "region": "Delhi",
  "country": "IN",
  "loc": "28.6519,77.2315",
  "org": "AS45271 Idea Cellular Limited",
  "postal": "110001",
  "timezone": "Asia/Kolkata",

In [4]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

# Specify the path to your CSV file
csv_file_path = "/Users/nithinkumar/Downloads/df1.csv"

# Read CSV into a DataFrame
df = spark.read.format("csv").option("header", "true").load(csv_file_path)

# Repartition the DataFrame to have a single partition
df = df.repartition(1)

# Specify the path where you want to save the Parquet file
parquet_output_path = "/Users/nithinkumar/Downloads/acs.parquet"

# Write the DataFrame to Parquet format
df.write.parquet(parquet_output_path)

# Stop the Spark session
spark.stop()
